In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [0]:
data= pd.read_csv('text_emotion.csv') #,encoding='ISO-8859-1', ISO-8859-1

In [17]:
data.head()

tweet_id  ...                                            content
0  1956967341  ...  @tiffanylue i know  i was listenin to bad habi...
1  1956967666  ...  Layin n bed with a headache  ughhhh...waitin o...
2  1956967696  ...                Funeral ceremony...gloomy friday...
3  1956967789  ...               wants to hang out with friends SOON!
4  1956968416  ...  @dannycastillo We want to trade with someone w...

[5 rows x 4 columns]

In [18]:
data = data[['content','sentiment']]

data = data[data.sentiment != "empty"]
data = data[data.sentiment != "neutral"]
data['content'] = data['content'].apply(lambda x: x.lower())
data['content'] = data['content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'positive'].size)
print(data[ data['sentiment'] == 'negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['content'].values)
X = tokenizer.texts_to_sequences(data['content'].values)
X = pad_sequences(X)

31176
29894


In [19]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 32, 128)           1280000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 32, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 1,535,194
Trainable params: 1,535,194
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

print(X_train[0])
print(X_test[0])
print(Y_train[0])
print(Y_test[0])

(20458, 32) (20458, 2)
(10077, 32) (10077, 2)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0  160 1059 5642    5  504    2    5
   48 3582  596  305]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0  379   30    4   68   31   22  356   35
 1059    3   29  506]
[1 0]
[0 1]


In [21]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Epoch 1/7
 - 55s - loss: 0.5883 - acc: 0.6864
Epoch 2/7
 - 53s - loss: 0.4837 - acc: 0.7729
Epoch 3/7
 - 53s - loss: 0.4218 - acc: 0.8101
Epoch 4/7
 - 53s - loss: 0.3662 - acc: 0.8410
Epoch 5/7
 - 53s - loss: 0.3171 - acc: 0.8656
Epoch 6/7
 - 53s - loss: 0.2745 - acc: 0.8861
Epoch 7/7
 - 53s - loss: 0.2417 - acc: 0.9005
score: 0.79
acc: 0.69


In [22]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 74.80314960629921 %
neg_acc 63.27913279132791 %


In [0]:
syriadata= pd.read_csv('Test.csv') #,encoding='ISO-8859-1', ISO-8859-1
# Keeping only the neccessary columns
syriadata = syriadata[['content']]
syriadata['content'] = syriadata['content'].apply(lambda x: x.lower())
syriadata['content'] = syriadata['content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

syriadata.head()

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(syriadata)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=32, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]

result=[]
for item in syriadata['content']:
    #vectorizing the tweet by the pre-fitted tokenizer instance
    #print(item)
    Ptwt = tokenizer.texts_to_sequences(item)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    Ptwt = pad_sequences(Ptwt, maxlen=32, dtype='int32', value=0)    
    Psentiment = model.predict(Ptwt,batch_size=1,verbose = 2)[0]
    print(item,Psentiment)
    result.append([item,Psentiment])

   

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 2877]]
ammoxxx great day woman did you notice the smile that never disappeared off my face ha murray all cos of you i owe you [0.4306574 0.5693426]
off to liverpool now [0.3172383  0.68276167]
sportday with charlotte [0.5753371  0.42466292]
roar_fear_me doing very well thank you very much  spending a lot of time behind the pc but with a focus and determination [0.9295662  0.07043383]
simbaaa mahimaaa sorry i couldnt help it twas to try and get if on the trending topics [0.5753371  0.42466292]
aanneeb ok  i kinda thought so i mean those 2  it would be funny though [0.4306574 0.5693426]
hoedown throwdown all i can say is im getting there [0.31242585 0.68757415]
sleeeping  just had to say happy momdukes day [0.5753371  0.42466292]
had an amazing time dancing to dj dan at ruby skye tonight and im still looking for that roommate in 

In [0]:
Rdf=pd.DataFrame(result)
Rdf[2]="NA"
depression=[]

for index, row in Rdf.iterrows():
    print(row[1])
    if(np.argmax(row[1]) == 0):
        print("negative")
        depression.append("negative")
    elif (np.argmax(row[1]) == 1):
        print("positive")
        depression.append("positive")

Rdf[2]=depression
Rdf.columns=["content","%sentiment","sentiment"]
Rdf.to_csv("output.csv") 
